In [43]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.utils.class_weight import compute_class_weight
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense

# Set random seed for reproducibility
np.random.seed(21)
random.seed(21)
tf.random.set_seed(21)

In [44]:
# Read in the independent and target variable datasets

stock = 'AAPL'
window_size = 10
pred_horizon = 5

X = pd.read_parquet(stock + '_X_' + str(window_size) + 'D.gzip')
y = pd.read_parquet(stock + '_y_' + str(pred_horizon) + 'D.gzip')
y = y.cumsum(axis=1)
y = y.iloc[:, -1:]
y_binary = (y > 0).astype(int)

In [45]:
# Split the dataset into training, validation and test datasets

num_features = 20

q_80 = int(len(X) * .8)
q_90 = int(len(X) * .9)

X_train, y_train = X[:q_80].to_numpy(), y_binary[:q_80].to_numpy()
X_val, y_val = X[q_80:q_90].to_numpy(), y_binary[q_80:q_90].to_numpy()
X_test, y_test = X[q_90:].to_numpy(), y_binary[q_90:].to_numpy()

X_train = X_train.reshape((-1, window_size, num_features))
X_val = X_val.reshape((-1, window_size, num_features))
X_test = X_test.reshape((-1, window_size, num_features))

In [46]:
# Define the early stopping callback to be used in all neural networks

early_stop = EarlyStopping(
    monitor='val_accuracy',        
    patience=5,                # wait 5 epochs for improvement
    restore_best_weights=True  # roll back to best weights
)

# Calculate class weights from training data

y_train_1D = y_train.reshape(-1)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_1D),
    y=y_train_1D
)
class_weights = dict(enumerate(class_weights))

In [47]:
# Fit an RNN model to the dataset

model_RNN = Sequential([
    SimpleRNN(16, 
         input_shape=(window_size, num_features)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_RNN.compile(loss='binary_crossentropy', 
              optimizer=RMSprop(learning_rate=0.001, rho=0.9),
              metrics=['accuracy'])

model_RNN.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, class_weight=class_weights, callbacks=[early_stop])

Epoch 1/100


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.5105 - loss: 0.6929 - val_accuracy: 0.5329 - val_loss: 0.6914
Epoch 2/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5256 - loss: 0.6908 - val_accuracy: 0.5377 - val_loss: 0.6928
Epoch 3/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5330 - loss: 0.6903 - val_accuracy: 0.5377 - val_loss: 0.6935
Epoch 4/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5341 - loss: 0.6900 - val_accuracy: 0.5353 - val_loss: 0.6947
Epoch 5/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5350 - loss: 0.6895 - val_accuracy: 0.5317 - val_loss: 0.6960
Epoch 6/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5372 - loss: 0.6891 - val_accuracy: 0.5246 - val_loss: 0.6978
Epoch 7/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5366 - loss: 0.6886 - val_accuracy: 0.5269 - val_loss: 0.6989


In [48]:
# Fit an LSTM model to the dataset

model_LSTM = Sequential([
    LSTM(16, 
         input_shape=(window_size, num_features)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_LSTM.compile(loss='binary_crossentropy', 
              optimizer=RMSprop(learning_rate=0.001, rho=0.9),
              metrics=['accuracy'])

model_LSTM.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, class_weight=class_weights, callbacks=[early_stop])

Epoch 1/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.5377 - loss: 0.6906 - val_accuracy: 0.5150 - val_loss: 0.6987
Epoch 2/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5305 - loss: 0.6899 - val_accuracy: 0.5198 - val_loss: 0.6982
Epoch 3/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5319 - loss: 0.6893 - val_accuracy: 0.5246 - val_loss: 0.6981
Epoch 4/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5409 - loss: 0.6888 - val_accuracy: 0.5210 - val_loss: 0.6984
Epoch 5/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5422 - loss: 0.6883 - val_accuracy: 0.5186 - val_loss: 0.6981
Epoch 6/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5438 - loss: 0.6878 - val_accuracy: 0.5210 - val_loss: 0.6981
Epoch 7/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5528 - loss: 0.6873 - val_accuracy: 0.5078 - val_loss: 0.6987
Epoch 8/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5466 - loss: 0.6866 - val_acc

In [49]:
# Fit a GRU model to the dataset

model_GRU = Sequential([
    GRU(16, 
         input_shape=(window_size, num_features)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_GRU.compile(loss='binary_crossentropy', 
              optimizer=RMSprop(learning_rate=0.001, rho=0.9),
              metrics=['accuracy'])

model_GRU.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, class_weight=class_weights, callbacks=[early_stop])

Epoch 1/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.5209 - loss: 0.6955 - val_accuracy: 0.5174 - val_loss: 0.6951
Epoch 2/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5331 - loss: 0.6910 - val_accuracy: 0.5162 - val_loss: 0.6971
Epoch 3/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5340 - loss: 0.6903 - val_accuracy: 0.5186 - val_loss: 0.6980
Epoch 4/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5358 - loss: 0.6899 - val_accuracy: 0.5186 - val_loss: 0.6981
Epoch 5/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5380 - loss: 0.6897 - val_accuracy: 0.5090 - val_loss: 0.6984
Epoch 6/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5383 - loss: 0.6895 - val_accuracy: 0.5114 - val_loss: 0.6985
Epoch 7/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5390 - loss: 0.6894 - val_accuracy: 0.5114 - val_loss: 0.6983
Epoch 8/100
209/209 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5394 - loss: 0.6893 - val

In [55]:
def evaluate_classifier(y_true, y_probs, model_name='Model', threshold=0.5):

    # Generate binary predictions based on threshold
    y_pred = (y_probs > threshold).astype(int)

    # Classification report
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

    # AUC
    auc = roc_auc_score(y_true, y_probs)

    # Build a summary row
    metrics = {
        "Model": model_name,
        "Accuracy": report["accuracy"],
        "Precision (0)": report["0"]["precision"],
        "Recall (0)": report["0"]["recall"],
        "F1 (0)": report["0"]["f1-score"],
        "Precision (1)": report["1"]["precision"],
        "Recall (1)": report["1"]["recall"],
        "F1 (1)": report["1"]["f1-score"],
        "AUC": auc
    }

    df_metrics = pd.DataFrame([metrics])
    df_metrics = df_metrics.apply(lambda x: round(x, 4))
    return df_metrics

In [56]:
# Predict the probabilities for the test dataset using the RNN model

y_RNN = model_RNN.predict(X_test)

fpr_RNN, tpr_RNN, thresholds_RNN = roc_curve(y_test, y_RNN)

# Choose threshold that gives better trade-off
J_RNN = tpr_RNN - fpr_RNN
optimal_idx_RNN = np.argmax(J_RNN)
optimal_threshold_RNN = thresholds_RNN[optimal_idx_RNN]

# Recalculate predictions
y_RNN_opt = (y_RNN > optimal_threshold_RNN).astype(int)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [57]:
# Predict the probabilities for the test dataset using the LSTM model

y_LSTM = model_LSTM.predict(X_test)

fpr_LSTM, tpr_LSTM, thresholds_LSTM = roc_curve(y_test, y_LSTM)

# Choose threshold that gives better trade-off
J_LSTM = tpr_LSTM - fpr_LSTM
optimal_idx_LSTM = np.argmax(J_LSTM)
optimal_threshold_LSTM = thresholds_LSTM[optimal_idx_LSTM]

# Recalculate predictions
y_LSTM_opt = (y_LSTM > optimal_threshold_LSTM).astype(int)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [58]:
# Predict the probabilities for the test dataset using the GRU model

y_GRU = model_GRU.predict(X_test)

fpr_GRU, tpr_GRU, thresholds_GRU = roc_curve(y_test, y_GRU)

# Choose threshold that gives better trade-off
J_GRU = tpr_GRU - fpr_GRU
optimal_idx_GRU = np.argmax(J_GRU)
optimal_threshold_GRU = thresholds_GRU[optimal_idx_GRU]

# Recalculate predictions
y_GRU_opt = (y_GRU > optimal_threshold_GRU).astype(int)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [59]:
results_RNN = evaluate_classifier(y_test, y_RNN_opt, model_name='RNN', threshold=optimal_threshold_RNN)
results_LSTM = evaluate_classifier(y_test, y_LSTM_opt, model_name='LSTM', threshold=optimal_threshold_LSTM)
results_GRU = evaluate_classifier(y_test, y_GRU_opt, model_name='GRU', threshold=optimal_threshold_GRU)

all_results = pd.concat([results_RNN, results_LSTM, results_GRU])
display(all_results)


,Model,Accuracy,Precision (0),Recall (0),F1 (0),Precision (1),Recall (1),F1 (1),AUC
0,RNN,0.4647,0.4405,0.9328,0.5984,0.6962,0.1151,0.1975,0.5239
0,LSTM,0.5138,0.4580,0.7479,0.5681,0.6429,0.3389,0.4438,0.5434
0,GRU,0.4359,0.4300,0.9804,0.5978,0.6667,0.0293,0.0561,0.5048
